In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'prepare/mesolitica-tpu.json'

In [2]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-tpu-general')

In [3]:
best = '346800'
!rm -rf t5-large-summary
!mkdir t5-large-summary
model = best

# gs://mesolitica-tpu-general/t5-large-summary/model.ckpt-356000.data-00001-of-00002

In [4]:
blob = bucket.blob(f't5-large-summary/model.ckpt-{model}.data-00000-of-00002')
blob.download_to_filename(f't5-large-summary/model.ckpt-{model}.data-00000-of-00002')

blob = bucket.blob(f't5-large-summary/model.ckpt-{model}.data-00001-of-00002')
blob.download_to_filename(f't5-large-summary/model.ckpt-{model}.data-00001-of-00002')

blob = bucket.blob(f't5-large-summary/model.ckpt-{model}.index')
blob.download_to_filename(f't5-large-summary/model.ckpt-{model}.index')

blob = bucket.blob(f't5-large-summary/model.ckpt-{model}.meta')
blob.download_to_filename(f't5-large-summary/model.ckpt-{model}.meta')

blob = bucket.blob('t5-large-summary/checkpoint')
blob.download_to_filename('t5-large-summary/checkpoint')

blob = bucket.blob('t5-large-summary/operative_config.gin')
blob.download_to_filename('t5-large-summary/operative_config.gin')

In [5]:
with open('t5-large-summary/checkpoint', 'w') as fopen:
    fopen.write(f'model_checkpoint_path: "model.ckpt-{model}"')

In [6]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
import t5

In [8]:
model = t5.models.MtfModel(
    model_dir='t5-large-summary',
    tpu=None,
    tpu_topology=None,
    model_parallelism=2,
    batch_size=5,
    sequence_length={"inputs": 1024, "targets": 1024},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=3,
    iterations_per_loop=100,
    mesh_shape="model:1,batch:1", 
    mesh_devices=["gpu:0"]
)

In [9]:
!rm -rf output/*

In [10]:
saved_model_path = model.export(
    'output',
    checkpoint_step=-1,
    beam_size=1,
    temperature=0.05,
    sentencepiece_model_path='sp10m.cased.t5.model'
)

INFO:tensorflow:Using config: {'_model_dir': 't5-large-summary', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3a536d5198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, 

INFO:tensorflow:Variable decoder/block_002/layer_000/SelfAttention/k                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_002/layer_000/SelfAttention/o                  size 1048576      slice_size 1048576      Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable decoder/block_002/layer_000/SelfAttention/q                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_002/layer_000/SelfAttention/v                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_002/layer_000/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable decoder/block_002/layer_001/En

INFO:tensorflow:Variable decoder/block_005/layer_001/EncDecAttention/k                size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_005/layer_001/EncDecAttention/o                size 1048576      slice_size 1048576      Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable decoder/block_005/layer_001/EncDecAttention/q                size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_005/layer_001/EncDecAttention/v                size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_005/layer_001/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable decoder/block_005/layer_002/De

INFO:tensorflow:Variable decoder/block_008/layer_002/DenseReluDense/wi/kernel         size 4194304      slice_size 4194304      Shape[d_model=1024, d_ff=4096]                              
INFO:tensorflow:Variable decoder/block_008/layer_002/DenseReluDense/wo/kernel         size 4194304      slice_size 4194304      Shape[d_ff=4096, d_model=1024]                              
INFO:tensorflow:Variable decoder/block_008/layer_002/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable decoder/block_009/layer_000/SelfAttention/k                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_009/layer_000/SelfAttention/o                  size 1048576      slice_size 1048576      Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable decoder/block_009/layer_000/Se

INFO:tensorflow:Variable decoder/block_012/layer_000/SelfAttention/q                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_012/layer_000/SelfAttention/v                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_012/layer_000/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable decoder/block_012/layer_001/EncDecAttention/k                size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable decoder/block_012/layer_001/EncDecAttention/o                size 1048576      slice_size 1048576      Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable decoder/block_012/layer_001/En

...

INFO:tensorflow:Variable encoder/block_002/layer_000/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable encoder/block_002/layer_001/DenseReluDense/wi/kernel         size 4194304      slice_size 4194304      Shape[d_model=1024, d_ff=4096]                              
INFO:tensorflow:Variable encoder/block_002/layer_001/DenseReluDense/wo/kernel         size 4194304      slice_size 4194304      Shape[d_ff=4096, d_model=1024]                              
INFO:tensorflow:Variable encoder/block_002/layer_001/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable encoder/block_003/layer_000/SelfAttention/k                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_003/layer_000/Se

INFO:tensorflow:Variable encoder/block_008/layer_000/SelfAttention/k                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_008/layer_000/SelfAttention/o                  size 1048576      slice_size 1048576      Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable encoder/block_008/layer_000/SelfAttention/q                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_008/layer_000/SelfAttention/v                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_008/layer_000/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable encoder/block_008/layer_001/De

INFO:tensorflow:Variable encoder/block_013/layer_000/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable encoder/block_013/layer_001/DenseReluDense/wi/kernel         size 4194304      slice_size 4194304      Shape[d_model=1024, d_ff=4096]                              
INFO:tensorflow:Variable encoder/block_013/layer_001/DenseReluDense/wo/kernel         size 4194304      slice_size 4194304      Shape[d_ff=4096, d_model=1024]                              
INFO:tensorflow:Variable encoder/block_013/layer_001/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable encoder/block_014/layer_000/SelfAttention/k                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_014/layer_000/Se

INFO:tensorflow:Variable encoder/block_019/layer_000/SelfAttention/k                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_019/layer_000/SelfAttention/o                  size 1048576      slice_size 1048576      Shape[heads=1024, d_model=1024]                             
INFO:tensorflow:Variable encoder/block_019/layer_000/SelfAttention/q                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_019/layer_000/SelfAttention/v                  size 1048576      slice_size 1048576      Shape[d_model=1024, heads=1024]                             
INFO:tensorflow:Variable encoder/block_019/layer_000/layer_norm/scale                 size 1024         slice_size 1024         Shape[d_model=1024]                                         
INFO:tensorflow:Variable encoder/block_019/layer_001/De

 variables/trainable: 7.38e+08
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from t5-large-summary/model.ckpt-346800
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: output/temp-b'1602516541'/saved_model.pb


In [11]:
saved_model_path.decode()

'output/1602516541'

In [12]:
import tensorflow_text  

tf.compat.v1.reset_default_graph()
sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], saved_model_path.decode())
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/1602516541/variables/variables


In [13]:
string = """
Amanah Kedah berpendapat jika ada Adun Pakatan Harapan atau Bersatu negeri itu mahu berpaling tadah memberikan sokongan kepada kumpulan Muafakat Nasional, mereka perku membuat kenyataan rasmi mengenainya.

Pengerusi Amanah Kedah, Phahrolrazi Mohd Zawawi, berkata disebabkan tiada mana-mana Adun membuat kenyataan berhubung isu itu maka kerajaan negeri berpendapat tiada apa-apa yang berlaku.

Ditemui media selepas mengadakan pertemuan tertutup lebih sejam dengan Menteri Besar, Mukhriz Mahathir, hari ini Phahrolrazi berkata pihaknya juga mendapati kerajaan negeri masih berfungsi seperti biasa.

"Kami bincang keadaan semasa, ada juga kita sentuh (cubaan menukar kerajaan negeri), tetapi kita lihat kerajaan masih berfungsi.

"Tidak ada apa-apa kenyataan dari pihak sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini," katanya seperti dipetik BH Online.

Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam ekoran berlaku pertemuan tertutup pemimpin PAS dan Umno Kedah di Alor Setar semalam.

Turut hadir Setiausaha Agung PAS yang juga Menteri di Jabatan Perdana Menteri, Takiyuddin Hassan, dan Menteri Besar Terengganu, Dr Ahmad Samsuri Mokhtar.  

Perkembangan itu berlaku kesan tindakan PKR memecat dan menggantung sejumlah anggota mereka baru-baru ini dan dipercayai memberi kesan terhadap pendirian wakil rakyat parti itu di Kedah.

Turut disebut-sebut akan beralih arah dalam perjalanan politik mereka ialah Adun Bersatu.

Untuk rekod berdasarkan pecahan parti PAS menguasai kerusi terbesar dalam DUN dan lazimnya pemimpin parti itu akan menjadi pilihan menjadi menteri besar jika berlaku pertukaran kerajaan.

Menurut Phahrolrazi, jika ada mana-mana wakil rakyat Bersatu atau PH mahu melompat, mereka wajar menyatakannya secara rasmi.

Tanpa kenyataan begitu, katanya, Amanah beranggapan isu perubahan kerajaan negeri masih bersifat spekulasi.

Timbalan Pengerusi Amanah Kedah, Dr Ismail Salleh, pula berkata ada kemungkinan Adun Bersatu, PH atau exco negeri tu yang sudah diumpan untuk membelakangkan mandat rakyat.

Beliau yang juga exco Kedah berkata memang sejak dulu lagi PAS cuba menjatuhkan kerajaan negeri dengan memujuk Adun PH serta Bersatu bertindak seperti rakan mereka di Perak, Johor dan Selangor.
"""

In [14]:
string1 = """
Timbalan Perdana Menteri Datuk Seri Dr Wan Azizah Wan Ismail mengakui pernah ditawarkan menjadi Perdana Menteri selepas Pakatan Harapan (PH) memenangi Pilihan Raya Umum ke-14 (PRU14) pada 9 Mei lepas.
Pertama kali berkongsi pengalaman berkenaan kepada kira-kira 200 rakyat Malaysia di Indonesia sempena lawatan kerja pertamanya ke negara itu, Dr Wan Azizah berkata Yang di-Pertuan Agong Sultan Muhammad V memberi tawaran itu memandangkan beliau ialah Presiden PKR, yang simbolnya digunakan semua parti dalam PH pada PRU14.
"Bagaimanapun, saya menolak tawaran itu kerana kita berpegang kepada janji bahawa Tun Dr Mahathir Mohamad akan menjadi Perdana Menteri dan saya menjadi Timbalan Perdana Menteri," katanya ketika berucap pada acara itu di Kedutaan Besar Malaysia di sini malam ini.
Mengakui PH tidak menyangka dapat menumbangkan Barisan Nasional (BN) yang memerintah selama 60 tahun, Dr Wan Azizah melahirkan rasa syukur pertukaran kerajaan selepas PRU14 dapat dijalankan secara demokratik dan aman.
Beliau, bagaimanapun berkata PH turut tidak menyangka kerajaan diambil alih, ditinggalkan dalam keadaan jumlah hutang begitu besar dan sistem yang parah.
Justeru, walaupun beberapa janji dalam manifesto PH pada PRU14 belum dapat ditunaikan, beliau memohon agar rakyat Malaysia supaya bersabar sebelum kerajaan membawa perubahan yang diinginkan rakyat.
Menjawab soalan daripada seorang diaspora Malaysia sama ada kerajaan akan menilai menteri-menteri baharu dan menukar mereka yang tidak menunjukkan prestasi baik, Dr Wan Azizah berkata masih terlalu awal untuk membuat penilaian memandangkan baharu lima bulan PH mengambil alih pentadbiran kerajaan.
"Beri kami sedikit masa kerana kami masih dalam 'learning curve' (keluk pembelajaran)," katanya sambil turut mengakui Perdana Menteri memberi amaran keras kepada jemaah menteri setiap kali mempengerusikan mesyuarat Kabinet supaya jangan sesekali terlibat dengan rasuah.
Menurutnya, hanya segelintir anggota jemaah menteri mempunyai pengalaman luas dalam urusan pentadbiran kerajaan iaitu Dr Mahathir dan Menteri Dalam Negeri Tan Sri Muhyiddin Yassin, manakala Menteri Kewangan Lim Guan Eng dan Menteri Hal Ehwal Ekonomi Datuk Seri Mohamed Azmin Ali berpengalaman dalam pentadbiran kerajaan negeri.
Dr Wan Azizah turut berkongsi kebanggaannya menjadi wanita pertama mempengerusikan mesyuarat Kabinet baru-baru ini, berikutan ketiadaan Dr Mahathir yang mengadakan lawatan ke luar negara.
Dr Wan Azizah mengadakan lawatan kerja sehari ke Indonesia pada Selasa, lawatan pertamanya ke negara ini selepas dilantik sebagai timbalan perdana menteri pada 21 Mei lepas.
"""

In [15]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(string)
string1 = cleaning(string1)

In [16]:
r = pred([f'ringkasan: {string}'] * 5)
r

array([b'"Tidak ada apa-apa kenyataan dari pihak-pihak di sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini," kata pengerusi Sppppliiit, pemimpin Umno dan pemimpin Umno telah bertemu di sebuah hotel di Alor Setar. Spekulasi mengenai pertukaran kerajaan telah menjadi panas sejak semalam',
       b'"Tidak ada apa-apa kenyataan dari pihak-pihak di sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini," kata pengerusi Sppppliiit, pemimpin Umno dan pemimpin Umno telah bertemu di sebuah hotel di Alor Setar. Spekulasi mengenai pertukaran kerajaan telah menjadi panas sejak semalam',
       b'"Tidak ada apa-apa kenyataan dari pihak-pihak di sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini," kata pengerusi Sppppliiit, pemimpin Umno dan pemimpin Umno telah bertemu di sebuah hotel di Alor Setar. Spekulasi mengenai pertukaran kerajaan telah menjadi panas sejak semalam',
       b'"Tidak ada apa-apa kenyataan dari pihak-pihak di sana (pembangkang) bahawa m

In [17]:
r = pred([f'ringkasan: {string1}'] * 5)
r

array([b'Dr Wan Azizah mengakui pernah ditawarkan menjadi Perdana Menteri. Beliau bercakap pada lawatan kerja ke Indonesia pada hari Selasa. Beliau berkata Perdana Menteri telah berjanji untuk menjadi perdana menteri',
       b'Dr Wan Azizah mengakui pernah ditawarkan menjadi Perdana Menteri. Beliau bercakap pada lawatan kerja ke Indonesia pada hari Selasa. Wan Azizah mengatakan dia tetap komited pada janji Perdana Menteri akan menjadi perdana menteri',
       b'Dr Wan Azizah mengakui pernah ditawarkan menjadi Perdana Menteri. Beliau bercakap pada lawatan kerja ke Indonesia pada hari Selasa. Beliau berkata Perdana Menteri telah berjanji untuk menjadi perdana menteri',
       b'Dr Wan Azizah mengakui pernah ditawarkan menjadi Perdana Menteri. Beliau bercakap pada lawatan kerja ke Indonesia pada hari Selasa. Beliau berkata Perdana Menteri telah berjanji untuk menjadi perdana menteri',
       b'Dr Wan Azizah mengakui pernah ditawarkan menjadi Perdana Menteri. Beliau bercakap pada lawatan 

In [18]:
# blob = bucket.blob(f't5-data/newstitle.tsv')
# blob.download_to_filename(f'newstitle.tsv')

import pandas as pd
df = pd.read_csv('newstitle.tsv', sep = '\t', error_bad_lines=False, header = None)

b'Skipping line 35779: expected 2 fields, saw 3\nSkipping line 69214: expected 2 fields, saw 3\nSkipping line 94511: expected 2 fields, saw 3\n'


In [19]:
r = pred([f'ringkasan: {df.iloc[0,0]}'] * 5)
r

array([b'- Sebuah organisasi polis tentera yang dihormati telah melangkah ke depan dengan memberikan bantuan kepada orang-orang yang terjejas oleh wabak Evida di Sekadau, Ohio, yang mempunyai masalah kewangan yang serius. Sukarelawan mengumpulkan sekitar 25 paket sembako, beras, tepung, minyak goreng, dan mie instan, kepada warga yang sebelumnya telah didata, lapor Mus. Sebilangan paket tersebut diserahkan secara langsung kepada warga yang sebelumnya telah didata, untuk mengelakkan kerumunan. "Semboyan kami adalah, \'Kami tidak Sedarah, tapi lebih dari Saudara,\'" kata seorang pegawai kumpulan itu. "Kekompakan dan rasa kemanusiaan kami tetap terjalin walaupun jarak yang jauh. " Dia mengatakan bahawa warga yang belum mendapat bantuan harus merasa "kewalahan ekonomi di tengah masa pandemi. " Anggota perkhidmatan sosial warga juga turut sama, memberikan sumbangan amal kepada masyarakat.',
       b'- Sebuah organisasi polis tentera yang baru dibentuk di Kabupaten Sekadau di Ohio melakukan 

In [20]:
df.iloc[1,0]

'Ketua MPR RI Bambang Soesatyo (Bamsoet) bersama Direksi PT Binabakti Niagaperkasa Andri Noviar, menyerahkan bantuan mobil ambulans kepada organisasi kemasyarakatan Gerakan Keadilan Bangun Solidaritas (GERAK BS). Nantinya, ambulans tersebut akan difungsikan untuk melayani kebutuhan masyarakat, khususnya yang kurang mampu secara gratis dalam menghadapi pandemi COVID-19. "Kebutuhan masyarakat terhadap ambulans sangat tinggi, apalagi di tengah pandemi COVID-19 yang membuat kesadaran masyarakat terhadap akses kesehatan meningkat. Bagi mereka yang kurang mampu, menyewa ambulans yang disiapkan rumah sakit menjadi beban tersendiri. Karena itu kehadiran GERAK BS yang mengoperasikan ambulans gratis untuk masyarakat, merupakan sebuah terobosan yang patut ditiru oleh berbagai lembaga organisasi kemasyarakatan lainnya," ujarnya dalam keterangan, Senin (13/4/2020). Bamsoet menilai, ambulans sudah memiliki standar pelayanan kesehatan yang memadai. Sehingga masyarakat tak perlu lagi ke rumah sakit me

In [21]:
r = pred([f'ringkasan: {df.iloc[1,0]}'] * 5)
r

array([b'- Ketua MPR RI Bambang Soesatyo memberikan penjelasan mengenai keputusan untuk menyewa ambulans kepada masyarakat yang kurang mampu, lapor Ap. "Kebutuhan masyarakat terhadap ambulans sangat tinggi, apalagi di tengah pandemi COVID-19 yang membuat kesadaran masyarakat terhadap akses kesehatan meningkat," katanya dalam keterangan yang dikeluarkan oleh Kepala Kepala MPR RI. "Bagi mereka yang kurang mampu, menyewa ambulans yang disiapkan rumah sakit menjadi beban tersendiri. " Dia menjelaskan bahawa ambulans sudah mempunyai standard perkhidmatan kesihatan yang memadai, sehingga masyarakat tidak perlu lagi ke rumah sakit untuk menggunakan transportasi yang kurang layak. "Tak semua warga memiliki kendaraan layak yang aman dan nyaman yang bisa digunakan sebagai transportasi dari dan ke rumah sakit," tambahnya. "Tak jarang berlaku, pasien yang berasal dari kalangan tidak mampu malah menuju rumah sakit dengan berjalan kaki. " Dia juga menyatakan bahawa ambulans dapat membantu memindahka

In [22]:
df.iloc[2,0]

'Bupati Purwakarta tak mewajibkan siswa dari tingkat ekonomi keluarga yang kurang mampu untuk menggunakan seragam sekolah. Kebijakan baru tersebut dikeluarkan bertepatan dengan dimulainya Kegiatan Belajar Mengajar (KBM) tahun ajaran 2017/2018. Aturan ini akan langsung diberlakukan pada tahun ajaran baru ini. "Tidak boleh ada paksaan bagi pelajar SD dan SMP di Purwakarta untuk mengenakan seragam sekolah jika kondisi ekonomi pelajar tersebut dinilai kurang mampu," ujar Bupati Purwakarta Dedi Mulyadi sebagaimana dikutip Antara, Selasa (18/7/2017). "Harus dilakukan identifikasi, berapa pelajar yang keadaan ekonominya mapan, dan berapa pelajar yang keadaan ekonominya rawan. Kita gotong-royong saja untuk membantu yang kurang mampu," imbuh Dedi. Ia juga menyerukan agar pihak-pihak dari keluarga mampu mau berempati kepada saudara-saudara di sekitarnya yang kurang mampu. Dedi mengaku akan sangat mengapresiasi jika ada warga dengan kondisi ekonomi mampu mau turut membantu warga-warga lain yang m

In [23]:
%%time

r = pred([f'ringkasan: {df.iloc[2,0]}'] * 5)
r

CPU times: user 30.7 s, sys: 5.12 s, total: 35.8 s
Wall time: 14.1 s


array([b'- Ibu bapa pelajar yang tidak mampu membeli seragam sekolah di Massachusetts tidak perlu mengikutinya -- dan tidak mahu anak-anak mereka memakainya. Pemerintah Kabupaten Purwakarta, yang memulai Kegiatan Belajar Mengajar (KBM) tahun ajaran 2017/2018, mengatakan langkah itu adalah sebagai tindak balas terhadap keluhan dari ibu bapa yang tidak mampu, lapor New York Times. "Harus dilakukan identifikasi, berapa pelajar yang keadaan ekonominya mapan, dan berapa pelajar yang keadaan ekonominya rawan," kata gubernur, Dedi Mulyadi, Selasa. "Kita gotong-royong saja untuk membantu yang kurang mampu. " Sekolah tersebut meminta pihak sekolah untuk melaksanakan langkah tersebut, dan peraturan itu akan terus dilaksanakan pada tahun ajaran baru. Minggu lalu, Pemerintah John Daly mengumumkan bahawa peraturan itu akan membolehkan anak-anak di daerah luar bandar memilih untuk tidak memilih, lapor Ap.',
       b'- Ibu bapa pelajar yang tidak mampu membeli seragam sekolah di Massachusetts tidak p

In [24]:
df.shape

(156683, 2)